<a href="https://colab.research.google.com/github/Jatin-Prakash/Deep-Learning/blob/main/Proj1_Reviews_Topic_Models_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tasks - 

1. Normalize case
2. Tokenize (using word_tokenize from NLTK)
3. POS tagging using the NLTK pos tagger
4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
 - Limit the data to only terms with these tags
5. Lemmatize (you want different forms of the terms to be treated as one, don't worry about providing POS tag to lemmatizer for now)
6. Remove stop words and punctuation (if there are any at all after the POS tagging)
7. Create a topic model using LDA on the cleaned up data with 12 topics
 - choose the topic model parameters carefully
 - what is the perplexity of the model?
 - what is the coherence of the model?
8. Analyze the topics, which pairs of topics can be combined?
9. Create topic model using LDA with what you think is the optimal number of topics
 - choose the topic model parameters carefully
 - is the perplexity better now?
 - is the coherence better now?
10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

#### Some code to get you started

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importing the usual utilities
import numpy as np, pandas as pd
import re, random, os, string

from pprint import pprint #pretty print
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from google.colab import files
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Reading in the reviews file

In [ ]:
upload = files.upload()

Saving K8 Reviews v0.2.csv to K8 Reviews v0.2.csv


In [ ]:
reviews0 = pd.read_csv("K8 Reviews v0.2.csv")
reviews0.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


## Task 1. Normalize case

In [ ]:
reviews_lower = [sent.lower() for sent in reviews0.review.values]
reviews_lower[0]

'good but need updates and improvements'

## Task 2. Tokenize (using word_tokenize from NLTK)

In [ ]:
reviews_token = [word_tokenize(sent) for sent in reviews_lower]
reviews_token[0]

['good', 'but', 'need', 'updates', 'and', 'improvements']

## Task 3. POS tagging using the NLTK pos tagger

Check out the POS tagging chapter from the NLTK book

http://www.nltk.org/book/ch05.html

Examples - 

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.pos_tag(reviews_token[0])

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [ ]:
sent = "I like to move it".split()
sent_tagged = nltk.pos_tag(sent)

In [ ]:
sent_tagged

[('I', 'PRP'), ('like', 'VBP'), ('to', 'TO'), ('move', 'VB'), ('it', 'PRP')]

In [ ]:
reviews_tagged = [nltk.pos_tag(tokens) for tokens in reviews_token]
reviews_tagged[0]

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [ ]:
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

## Task 4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
 - Limit the data to only terms with these tags


In [ ]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

You see that for each term, the POS taggger returns a tuple. The first element of the tuple being the term, the second being the tag.

In [ ]:
tagged_tuple = nltk.pos_tag(['great'])
tagged_tuple[0]

('great', 'JJ')

In [ ]:
pprint(tagged_tuple[0][0])
pprint(tagged_tuple[0][1])

'great'
'JJ'


In [ ]:
reviews_noun=[]
for sent in reviews_tagged:
    reviews_noun.append([token for token in sent if re.search("NN.*", token[1])])
reviews_noun[0]

[('updates', 'NNS'), ('improvements', 'NNS')]

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

You'll need to extract the tag from the resulting tuple, of course and then limit to the desired tags

## Task  5. Lemmatize
 - you want different forms of the terms to be treated as one
 - don't worry about providing POS tag to lemmatizer for now

In [ ]:
lemm = WordNetLemmatizer()
reviews_lemm=[]
for sent in reviews_noun:
    reviews_lemm.append([lemm.lemmatize(word[0]) for word in sent])

In [ ]:
reviews_lemm[0]

['update', 'improvement']

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Task  6. Remove stop words and punctuation (if there are any at all after the POS tagging)

Use NLTK standard stop word list and the punctuations

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")

In [ ]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + [".."]
reviews_sw_removed=[]
for sent in reviews_lemm:
    reviews_sw_removed.append([term for term in sent if term not in stop_updated])

In [ ]:
reviews_sw_removed[1]

['mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'charger',
 'hour']

## Task 7. Create a topic model using LDA on the cleaned up data with 12 topics
 - what is the coherence of the model?
 
 Use gensim for this task

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

In [ ]:
id2word = corpora.Dictionary(reviews_sw_removed)
texts = reviews_sw_removed
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[200])

[(426, 1), (427, 1), (428, 1), (429, 1)]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.182*"mobile" + 0.062*"charger" + 0.042*"battery" + 0.037*"time" + '
  '0.036*"heat" + 0.031*"charge" + 0.031*"hour" + 0.028*"turbo" + 0.027*"hr" + '
  '0.022*"charging"'),
 (1,
  '0.077*"feature" + 0.051*"speaker" + 0.048*"call" + 0.027*"handset" + '
  '0.019*"headphone" + 0.017*"power" + 0.016*"button" + 0.016*"note" + '
  '0.016*"sound" + 0.016*"earphone"'),
 (2,
  '0.031*"screen" + 0.027*"glass" + 0.020*"video" + 0.019*"app" + 0.019*"time" '
  '+ 0.017*"sensor" + 0.016*"key" + 0.015*"internet" + 0.015*"music" + '
  '0.015*"photo"'),
 (3,
  '0.047*"hai" + 0.038*"h" + 0.021*"ho" + 0.014*"plz" + 0.013*"hi" + '
  '0.011*"bhi" + 0.010*"k" + 0.008*"ye" + 0.008*"ko" + 0.008*"weight"'),
 (4,
  '0.184*"battery" + 0.083*"camera" + 0.074*"problem" + 0.055*"performance" + '
  '0.052*"phone" + 0.042*"backup" + 0.042*"heating" + 0.026*"issue" + '
  '0.024*"life" + 0.024*"day"'),
 (5,
  '0.106*"money" + 0.056*"value" + 0.017*"volta" + 0.015*"item" + '
  '0.015*"killer" + 0.011*"improveme

In [ ]:
#Leneovo Battery is quite aweful and it drains out pretty faster! I always prefer to use batteries with maximum capacity which can last upto 36 hours.

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5397275941208991


## Task 8. Analyze the topics, which pairs of topics can be combined?
 - you can assume that if a pair of topics has very similar top terms, they are very close and can be combined

### Looking at the topics and each terms following can be combined -

** Topic 2 and 5 possibly talks about 'pricing'  
 Topic 4, 6 and 10 closely talks about 'battery related issues'  
 Topic 3 and 11 vaguely talks about 'performance'**

## Task 9. Create topic model using LDA with what you think is the optimal number of topics

 - is the coherence better now?

In [ ]:
# Build LDA model
lda_model8 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

Printing the coherence of the model

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model8, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.580100755115897


## Task 10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

In [ ]:
x = lda_model8.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

In [ ]:
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

0::['mobile', 'charger', 'waste', 'money', 'turbo', 'box', 'superb', 'piece', 'ok', 'awesome']
1::['product', 'note', 'lenovo', 'k8', 'phone', 'service', 'feature', 'handset', 'issue', 'month']
2::['phone', 'screen', 'time', 'call', 'option', 'problem', 'day', 'note', 'lot', 'glass']
3::['hai', 'h', 'ho', 'set', 'speed', 'plz', 'hi', 'bhi', 'ye', 'ko']
4::['battery', 'phone', 'problem', 'camera', 'issue', 'performance', 'backup', 'heating', 'day', 'hour']
5::['amazon', 'delivery', 'product', 'return', 'customer', 'worth', 'money', 'policy', 'star', 'time']
6::['camera', 'quality', 'network', 'issue', 'speaker', 'phone', 'sim', 'mode', 'sound', 'money']
7::['phone', 'price', 'camera', 'feature', 'range', 'processor', 'performance', 'budget', 'android', 'ram']



In [ ]:
#possible topics from terms present

#Topic1 = product accessories
#Topic2 = amazon
#Topic3 = pricing
#Topic4 = phone performance
#Topic5 = battery related issues
#Topic6 = camera quality
#Topic7 = sound features
#Topic8 = overall general phone features 